In [123]:
import joblib
import re
import ast
gpt_preds = joblib.load('gpt_preds')
best_gpt_preds = joblib.load('best_gpt_preds')
y_true = joblib.load('y_true')
eps = joblib.load('eval_episodes')

In [124]:
len(best_gpt_preds), len(y_true), len(gpt_preds)

(401, 401, 401)

In [125]:
gpt_preds[0]

'{"CASE_NUMBER": ["CWP No.6780 of 1999"], "DATE": ["20.12.2001", "04.08.2009"], "PRECEDENT": ["Full Bench judgment in Vijay Sharma \'s case", "State of Punjab and another Vs . Surjit Singh and others ( 2009 ) 9 SCC 514"]}'

In [126]:
best_gpt_preds[0]

"{CASE_NUMBER - ['WP No.2157 of 1995', 'WP No.5650 of 1987'], PRECEDENT - ['Paupuk Kannu Anni v. Thoppayya Mudaliar']}"

In [128]:
print(y_true[0])

['CASE_NUMBER', 'CASE_NUMBER', 'CASE_NUMBER', 'CASE_NUMBER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'OTHER_PERSON', 'OTHER_PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATE', 'O', 'O', 'O', 'O', 'O', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'O']


In [131]:
preds = []
true = []
for i, p in enumerate(gpt_preds):
    dictionary = ast.literal_eval(p)
    preds.append(dictionary)
    true.append(y_true[i])

In [132]:
preds[0]

{'CASE_NUMBER': ['CWP No.6780 of 1999'],
 'DATE': ['20.12.2001', '04.08.2009'],
 'PRECEDENT': ["Full Bench judgment in Vijay Sharma 's case",
  'State of Punjab and another Vs . Surjit Singh and others ( 2009 ) 9 SCC 514']}

In [147]:
example_label = {'CASE_NUMBER': ['CWP No.6780 of 1999'],
 'DATE': ['20.12.2001', '04.08.2009'],
 'PRECEDENT': ["Full Bench judgment in Vijay Sharma 's case",
  'State of Punjab and another Vs . Surjit Singh and others ( 2009 ) 9 SCC 514']}

tokenized_text = ['CWP', 'No.6780', 'of', '1999', 'has', 'been', 'allowed', 'by', 'the', 'Division', 'Bench', 'on', '20.12.2001', 'again', 'on', 'the', 'basis', 'of', 'Full', 'Bench', 'judgment', 'in', 'Vijay', 'Sharma', "'s", 'case', '(', 'supra', ')', ',', 'but', 'the', 'said', 'judgment', 'has', 'been', 'set', 'aside', 'on', '04.08.2009', 'in', 'a', 'judgment', 'reported', 'as', 'State', 'of', 'Punjab', 'and', 'another', 'Vs', '.', 'Surjit', 'Singh', 'and', 'others', '(', '2009', ')', '9', 'SCC', '514', '.']
def structure_labels(example):
    new_labels = {k: [] for k in example}
    for k in example:
        for l in example[k]:
            new_labels[k].extend(l.split())
    
    return new_labels

In [149]:
print(structure_labels(example_label))

{'CASE_NUMBER': ['CWP', 'No.6780', 'of', '1999'], 'DATE': ['20.12.2001', '04.08.2009'], 'PRECEDENT': ['Full', 'Bench', 'judgment', 'in', 'Vijay', 'Sharma', "'s", 'case', 'State', 'of', 'Punjab', 'and', 'another', 'Vs', '.', 'Surjit', 'Singh', 'and', 'others', '(', '2009', ')', '9', 'SCC', '514']}


In [152]:
new_preds = []
for pred in preds:
    new_preds.append(structure_labels(pred))

print(new_preds[1])

{'WITNESS': ['PW2', 'HC', 'Vandana']}


In [168]:
# Provided entities dictionary and tokenized text
entities_dict = {'CASE_NUMBER': ['CWP', 'No.6780', 'of', '1999'],
                 'DATE': ['20.12.2001', '04.08.2009'],
                 'PRECEDENT': ['Full', 'Bench', 'judgment', 'in', 'Vijay', 'Sharma', "'s", 'case', 'State', 'of', 'Punjab', 'and', 'another', 'Vs', '.', 'Surjit', 'Singh', 'and', 'others', '(', '2009', ')', '9', 'SCC', '514']}
tokenized_text = ['CWP', 'No.6780', 'of', '1999', 'has', 'been', 'allowed', 'by', 'the', 'Division', 'Bench', 'on', '20.12.2001', 'again', 'on', 'the', 'basis', 'of', 'Full', 'Bench', 'judgment', 'in', 'Vijay', 'Sharma', "'s", 'case', '(', 'supra', ')', ',', 'but', 'the', 'said', 'judgment', 'has', 'been', 'set', 'aside', 'on', '04.08.2009', 'in', 'a', 'judgment', 'reported', 'as', 'State', 'of', 'Punjab', 'and', 'another', 'Vs', '.', 'Surjit', 'Singh', 'and', 'others', '(', '2009', ')', '9', 'SCC', '514', '.']

def tokenize_labels(label_dict, text):
    # Initialize labeled tokenized list
    labeled_tokenized_list = []

    # Iterate over tokenized text
    for token in text:
        token_label = 'O'

        for label, entities in label_dict.items():
            if token in entities:
                token_label = label
                break
        labeled_tokenized_list.append(token_label)
    return labeled_tokenized_list

In [169]:
print(tokenize_labels(entities_dict, tokenized_text))

['CASE_NUMBER', 'CASE_NUMBER', 'CASE_NUMBER', 'CASE_NUMBER', 'O', 'O', 'O', 'O', 'O', 'O', 'PRECEDENT', 'O', 'DATE', 'O', 'O', 'O', 'O', 'CASE_NUMBER', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'O', 'PRECEDENT', 'O', 'O', 'O', 'O', 'PRECEDENT', 'O', 'O', 'O', 'O', 'O', 'DATE', 'PRECEDENT', 'O', 'PRECEDENT', 'O', 'O', 'PRECEDENT', 'CASE_NUMBER', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT', 'PRECEDENT']


In [177]:
y_preds = []
for i, p in enumerate(new_preds):
    y_preds.append(tokenize_labels(p, eps[i]['query_set'][0]))

In [178]:
len(y_preds)

401

In [179]:
joblib.dump(y_preds, 'y_preds')

['y_preds']